# 🚀 **JLSatiro Clipper AI - V23.0 Titanium Edition**

Bem-vindo ao **JLSatiro Clipper AI**, a plataforma definitiva para criação de cortes virais automatizados.

### ⚡ **Recursos Titanium:**
- **Smart Crop V2.5**: Rastreamento facial ultra-preciso com MediaPipe (Fixed).
- **Hyper-Speed Transcription**: Transcrição GPU acelerada com Faster-Whisper.
- **Auto-Viral**: Geração automática de Thumbnails e Hooks.
- **100% Local**: Otimizado para rodar localmente no Colab com zero delay.

---

In [ ]:
#@title 1. 🛠️ Instalação do Sistema (Execute e Aguarde)
#@markdown Isso instalará todas as dependências, configurará o ambiente e corrigirá bugs conhecidos (ImageMagick, MediaPipe).

import os
import subprocess
import sys

print("💎 [JLSatiro] Iniciando Setup Titanium V23.0...")

# 1. Verificar GPU
print("🖥️ [1/4] Verificando Hardware...")
try:
    subprocess.run(["nvidia-smi"], check=True)
    print("   ✅ GPU NVIDIA Detectada! Modo Turbo Ativado.")
except:
    print("   ⚠️ AVISO: GPU não detectada. O sistema rodará em modo CPU (lento).")
    print("   👉 Dica: Vá em 'Runtime' > 'Change runtime type' > 'T4 GPU'.")

# 2. Instalar Dependências do Sistema (APT)
print("📦 [2/4] Instalando Motores de Sistema (FFmpeg, ImageMagick)...")
subprocess.run("apt-get update -qq", shell=True)
subprocess.run("apt-get install -y ffmpeg imagemagick libsndfile1 sox -qq", shell=True)

# CORREÇÃO: ImageMagick Policy p/ MoviePy
if os.path.exists("/etc/ImageMagick-6/policy.xml"):
    print("   🔧 Aplicando Patch no ImageMagick...")
    subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

# 3. Instalar Bibliotecas Python (PIP)
print("🐍 [3/4] Instalando Bibliotecas Python (Isso pode levar 2-3 min)...")

# GPU LLM Compilation (Critical for Speed)
print("   🧠 Compilando Llama.cpp com suporte CUDA (GPU)...")
subprocess.run('CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python --no-cache-dir', shell=True)

libraries = [
    # Core AI
    "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121",
    "faster-whisper>=1.0.0",
    "openai-whisper",
    "mediapipe>=0.10.14",
    "transformers",
    
    # LLM & Logic (Version Locked)
    "instructor>=0.5.0,<1.0.0",
    "pydantic",
    
    # Media Processing
    "moviepy",
    "opencv-python-headless",
    "librosa",
    "soundfile",
    "imageio",
    "imageio-ffmpeg",
    "pillow",
    "pydub",

    # Web & Utils
    "gradio>=4.0.0,<5.0.0",
    "flask",
    "flask-cors",
    "pytubefix>=6.0.0",
    "yt-dlp",
    "python-dotenv",
    "numpy<2.0.0",
    "pandas",
    "edge-tts"
]

pip_cmd = f"{sys.executable} -m pip install --upgrade --no-cache-dir"
for lib in libraries:
    subprocess.run(f"{pip_cmd} \"{lib}\"", shell=True, stdout=subprocess.DEVNULL)

print("   ✅ Dependências Python instaladas.")

# 4. Clonar Repositório e Configurar Workspace
print("📂 [4/4] Configurando Workspace...")
repo_url = "https://github.com/noviso123/JLSatiroClipperAI.git"
repo_dir = "/content/JLSatiroClipperAI"

if not os.path.exists(repo_dir):
    subprocess.run(f"git clone {repo_url} {repo_dir}", shell=True)
else:
    subprocess.run(f"cd {repo_dir} && git pull", shell=True)

# Criar pastas necessárias
os.makedirs(f"{repo_dir}/downloads", exist_ok=True)
os.makedirs(f"{repo_dir}/temp_work", exist_ok=True)
os.makedirs(f"{repo_dir}/backend/models", exist_ok=True)

print("\n✨ INSTALAÇÃO COMPLETA! Execute a próxima célula para iniciar.")

In [ ]:
#@title 2. 🚀 Iniciar JLSatiro Clipper AI
#@markdown Clique no play e aguarde o link público `https://xxxx.gradio.live` aparecer abaixo.

import os
import sys

# Garantir que estamos no diretório correto
os.chdir("/content/JLSatiroClipperAI")

# Forçar patch do MediaPipe caso o repo não tenha atualizado ainda (Backup Safety)
video_engine_path = "backend/video_engine.py"
if os.path.exists(video_engine_path):
    with open(video_engine_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    if "from mediapipe import solutions" in content:
        print("🔧 Aplicando correção hotfix no MediaPipe (Backup)...")
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face_detection = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face_detection = mp.solutions.face_detection"
        )
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face = mp.solutions.face_detection"
        )
        with open(video_engine_path, 'w', encoding='utf-8') as f:
            f.write(content)

# Executar App
!python app.py